### Pre-requisitos para que funcione correctamente
<ul>
    <li>pip install ibm_db</li>
    <li> Instalar nueva version DB2 11.5 del <a href="https://w3-01.ibm.com/download/standardsoftware/PC/lang_en/issiCatalogPC.html" target="_blank">pc@catalog</a></li>
    <li> Usar version modificada del .bat setup_cmd para setear las conexiones (la tengo yo)</li>
    <li> Carpeta utils, tiene que tener licencia 11.5 (la tengo yo) y los GSKit nuevos version 8.50.89 al 27/4</li>
    <li> El conection string, además de lo que figura en github --> <a href="https://github.com/ibmdb/python-ibmdb#api" target="_blank">IBM_DB API</a>, tiene que tener los parametros security=ssl,SSLClientKeystoredb=<b>Dir ibmca.kdb</b>;SSLClientKeystoreDBPassword=<b>Psw .kdb (.bat)</b></li>
    <li>hostname, database name, port datos claves que tienen que estar bien en ODBC</li>
</ul>

##### IBM_DB has proven not work for ICFS connection

In [3]:
import ibm_db
import ibm_db_dbi as dbi
import pandas as pd
import numpy as np
import time
import pickle


start = time.time()

inputfile=r"C:\Users\016434613\Desktop\Todo\input_PyDB2.txt"
datalist = []
SSLClientKeystoredb_path="C:\ProgramData\IBM\DB2\DB2COPY1\DB2\ibmca.kdb"


with open("cred_db","rb") as f:
    cred_dict = pickle.load(f)

uid=cred_dict['user']
psw=cred_dict['psw']
SSLClientKeystoredb=SSLClientKeystoredb_path
SSLClientKeystoreDBPassword=cred_dict['SSLClientKeystoreDBPassword']


''' Usando dbi, podemos usar el metodo de pandas read_sql y tener un dataframe directo del resultado '''

conn=dbi.connect("DATABASE=USIBMVRDP1H;HOSTNAME=usibmvrdp1h.ssiplex.pok.ibm.com;PORT=5520;PROTOCOL=TCPIP;UID="+uid+";PWD="+psw+";Security=SSL;SSLClientKeystoredb="+SSLClientKeystoredb+";SSLClientKeystoreDBPassword="+SSLClientKeystoreDBPassword+";", "", "")



sql = ''' SELECT CTY, DIV, LC, TOLI, MAJ, MINOR, SMIN, LERU, FID, SRC, ACCID, ACCTMO,
             ACCTYEAR, AMTLOC, AMTDLR, AMTLOC_REP, AMTDLR_REP, VOUCHER_NBR,
             OEM_PROD_ID, DESCR1, LOCFLD1
          FROM LRPCC.LADETAIL_PM_D
          WHERE (CTY = '613' AND MAJ = '601' AND LERU IN ('001977')
             AND LC IN ('01', '04')) '''



df2=pd.read_sql(sql,conn)

df_Minor = pd.read_excel(r"C:\Users\016434613\Desktop\Todo\Minor Codes.xlsx",sheet_name="REF (2)",converters={'Minor':str})
df_Minor = df_Minor.loc[:,['Minor','Desc']]
df_Minor = df_Minor.rename(columns={'Minor':'MINOR'})
df2 = df2.merge(df_Minor, on="MINOR", how="left")
df2.to_excel(r"C:\Users\016434613\Desktop\dept1977_exp.xlsx", index=False)



end = time.time()
mins = (end-start)/60.00
if mins < 1:
    mins = end-start
    minsstr = "Report created in: {:.2f} seconds".format(mins) 
    print(minsstr)
else:
    minsstr = "Report created in: {:.2f} minutes".format(mins)
    print(minsstr)

Report created in: 5.07 seconds


In [4]:
table_ = pd.pivot_table(df2, values=['AMTDLR_REP'], index=['Desc'], columns=['ACCTMO'],
                       aggfunc=np.sum, fill_value=0, margins=True, margins_name="Total")
table_.sort_values(by=('AMTDLR_REP','Total'),ascending=False)

AMTDLR_REP            \
ACCTMO                                                     01        02   
Desc                                                                      
Total                                                52737.18  46393.62   
Salaries                                             26228.57  25791.72   
Social Seurity Taxes                                 10618.83   8243.68   
Vacations Accrual                                     9434.73    487.60   
Bonuses                                               2617.52   2738.05   
Welfare                                                855.10   1705.31   
Variable Pay                                             0.00   4760.46   
Awards                                                1299.80   1195.41   
Other Comp                                             675.77    723.12   
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...     171.46   -171.46   
DEFINED CONTRIBUTION EXPENSE                           339.86    323.68   
Overtime                                                 0.00      0.00   
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS             227.63    228.67   
Deprec IBM equip-Non Wkst                              163.00    198.00   
Servicios IT - IGA SD                                    0.00      0.00   
NON-WIRELESS REMOTE DSL ACCESS                          38.95     95.62   
Postretirement Plan                                     61.88     68.07   
Retirement Plan                                          4.08      3.88   
Audio Conferencing Products and Services                 0.00      1.81   

                                                                        \
ACCTMO                                                    03        04   
Desc                                                                     
Total                                               44815.14  55955.78   
Salaries                                            23798.32  27783.80   
Social Seurity Taxes                                 8630.75   6013.82   
Vacations Accrual                                    3958.21  12342.50   
Bonuses                                              2729.37   3811.74   
Welfare                                               958.10   1838.78   
Variable Pay                                            0.00      0.00   
Awards                                               1317.13    347.79   
Other Comp                                            682.16    718.88   
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...   1498.74    356.29   
DEFINED CONTRIBUTION EXPENSE                          291.49    412.91   
Overtime                                              383.45    566.93   
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS            241.51    246.14   
Deprec IBM equip-Non Wkst                             198.00    308.00   
Servicios IT - IGA SD                                   0.00    826.32   
NON-WIRELESS REMOTE DSL ACCESS                         56.34    308.86   
Postretirement Plan                                    68.07     68.07   
Retirement Plan                                         3.50      4.95   
Audio Conferencing Products and Services                0.00      0.00   

                                                               
ACCTMO                                                  Total  
Desc                                                           
Total                                               199901.72  
Salaries                                            103602.41  
Social Seurity Taxes                                 33507.08  
Vacations Accrual                                    26223.04  
Bonuses                                              11896.68  
Welfare                                               5357.29  
Variable Pay                                          4760.46  
Awards                                                4160.13  
Other Comp                                            2799.93  
TRA

#### Para que la conexión de PyOdbc funcione, tuve que cambiar la psw con el config assistant de DB2 y antes tenía una con mayusculas y chars especiales como _ y la cambie por una toda en minusculas sin chars especiales


##### PyODBC me funciono para ICFS mientras que IBM_DB no

In [1]:
import pyodbc
import pandas as pd
import pickle
import time
import os

start = time.time()

#file should be store in the same path
cred_file = "cred_db"
minor_codes = r"C:\Users\016434613\Desktop\Todo\Minor Codes.xlsx"
outfile = r"C:\Users\016434613\Desktop\dept1977_exp.xlsx"

if os.path.exists(cred_file):
    with open(cred_file,"rb") as f:
        cred_dict = pickle.load(f)


try:
    conn_string="DSN=DP1H;UID={};PWD={}".format(cred_dict['user'],cred_dict['psw'])
except:
    print("File cred_db does not exists")
conn = pyodbc.connect(conn_string)

sql = ''' SELECT CTY, DIV, LC, TOLI, MAJ, MINOR, SMIN, LERU, FID, SRC, ACCID, ACCTMO,
             ACCTYEAR, AMTLOC, AMTDLR, AMTLOC_REP, AMTDLR_REP, VOUCHER_NBR,
             OEM_PROD_ID, DESCR1, LOCFLD1
          FROM LRPCC.LADETAIL_PM_D
          WHERE (CTY = '613' AND MAJ = '601' AND LERU IN ('001977')
             AND LC IN ('01', '04')) '''

df =pd.read_sql(sql,conn)

end = time.time()
mins = (end-start)/60.00
if mins < 1:
    mins = end-start
    minsstr = "Report created in: {:.2f} seconds".format(mins) 
    print(minsstr)
else:
    minsstr = "Report created in: {:.2f} minutes".format(mins)
    print(minsstr)

df_Minor = pd.read_excel(minor_codes,sheet_name="REF (2)",converters={'Minor':str})
df_Minor = df_Minor.loc[:,['Minor','Desc']]
df_Minor = df_Minor.rename(columns={'Minor':'MINOR'})
df = df.merge(df_Minor, on="MINOR", how="left")
df.to_excel(outfile, index=False)

Report created in: 3.97 seconds


,CTY,DIV,LC,TOLI,MAJ,MINOR,SMIN,LERU,FID,SRC,...,ACCTMO,ACCTYEAR,AMTLOC,AMTDLR,AMTLOC_REP,AMTDLR_REP,VOUCHER_NBR,OEM_PROD_ID,DESCR1,LOCFLD1
0,613,02,01,L,601,0100,0000,001977,095,094,...,01,2022,272734159.0,2675963.0,272734159.0,2622857.0,GVP0613,,GPIH,
1,613,02,01,L,601,0100,0000,001977,095,094,...,02,2022,274321613.0,2637961.0,274321613.0,2579172.0,GVP0613,,GPIH,
2,613,02,01,L,601,0100,0000,001977,095,094,...,03,2022,260338468.0,2447480.0,260338468.0,2379832.0,GVP0613,,GPIH,
3,613,02,01,L,601,0100,0000,001977,095,094,...,04,2022,314745016.0,2877011.0,314745016.0,2778380.0,GVP0613,,GPIH,
4,613,02,01,L,601,0140,0000,001977,095,094,...,01,2022,3803085.0,37314.0,3803085.0,36574.0,GVP0613,,GPIH,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,613,02,04,L,601,9878,0000,001977,994,694,...,04,2022,20145827.0,185503.0,20145827.0,185503.0,0000735,,CIO Local WS Inventory,
504,613,02,04,L,601,9878,0000,001977,994,694,...,04,2022,20145827.0,185503.0,20145827.0,185503.0,0000735,,CIO Local WS Inventory,
505,613,02,04,L,601,9878,0000,001977,994,694,...,04,2022,20145827.0,185503.0,20145827.0,185503.0,0000735,,CIO Local WS Inventory,
506,613,02,04,L,601,9878,0000,001977,994,694,...,04,2022,20145827.0,185503.0,20145827.0,185503.0,0000735,,CIO Local WS Inventory,
